Here I only test parts of the code before passing them on to the main notebook.

In [108]:
import time
import requests
from dotenv import load_dotenv
import os
import folium
import pandas as pd

In [109]:
m = folium.Map(location=[20.9939879883004, -89.62853393602846], zoom_start=12)
delta = 0.0135 

initial_lat, initial_lng = 20.944451,-89.654108 


rectangles_viewports = []

for i in range(4):
    for j in range(4):
        low = [initial_lat+i*delta, initial_lng+j*delta]
        high = [initial_lat+(i+1)*delta, initial_lng+(j+1)*delta]
        folium.Rectangle(
            bounds = [low, high],
            tooltip = f'({i+1},{j+1})',
            fill = True
        ).add_to(m)
        
        rectangles_viewports.append((tuple(low), tuple(high)))
m

In [110]:
print(rectangles_viewports)

[((20.944451, -89.654108), (20.957951, -89.640608)), ((20.944451, -89.640608), (20.957951, -89.62710799999999)), ((20.944451, -89.62710799999999), (20.957951, -89.613608)), ((20.944451, -89.613608), (20.957951, -89.60010799999999)), ((20.957951, -89.654108), (20.971451000000002, -89.640608)), ((20.957951, -89.640608), (20.971451000000002, -89.62710799999999)), ((20.957951, -89.62710799999999), (20.971451000000002, -89.613608)), ((20.957951, -89.613608), (20.971451000000002, -89.60010799999999)), ((20.971451000000002, -89.654108), (20.984951000000002, -89.640608)), ((20.971451000000002, -89.640608), (20.984951000000002, -89.62710799999999)), ((20.971451000000002, -89.62710799999999), (20.984951000000002, -89.613608)), ((20.971451000000002, -89.613608), (20.984951000000002, -89.60010799999999)), ((20.984951000000002, -89.654108), (20.998451, -89.640608)), ((20.984951000000002, -89.640608), (20.998451, -89.62710799999999)), ((20.984951000000002, -89.62710799999999), (20.998451, -89.613608

In [ ]:
load_dotenv()
API_KEY = os.getenv('API_KEY')

url = "https://places.googleapis.com/v1/places:searchText"
headers = {
    'Content-Type' : 'application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask': 'places.id,places.displayName,places.formattedAddress,places.primaryTypeDisplayName,places.viewport'
}

In [ ]:
raw_data = []
for low, high in rectangles_viewports:
    try: 
        json = {
            'textQuery' : 'cafeteria',
            'includedType': 'cafe',
            'strictTypeFiltering':True,
            'locationRestriction' : {
                'rectangle':{
                    'low':{
                        'latitude' : low[0],
                        'longitude' : low[1]
                    },
                    'high':{
                        'latitude': high[0],
                        'longitude': high[1]
                        
                    }
                }
            }
        }
        
        response = requests.post(url = url, json=json, headers = headers)
        response.raise_for_status()
        
        data = response.json()
        
        raw_data.append(data)
        
    except requests.exceptions.RequestException as e:
        print(f'ERROR!!! --> {e}')
    
    time.sleep(0.5)
    

In [114]:
raw_df = pd.concat([pd.DataFrame(place['places']) for place in raw_data if len(place)!=0], ignore_index=True)
raw_df

,id,formattedAddress,viewport,displayName,primaryTypeDisplayName
0,ChIJlSpZ2wlzVo8RdsMWLOZqrg8,"Aviación 709, Nueva Sambulá, 97250 Mérida, Yuc...","{'low': {'latitude': 20.946772919708497, 'long...","{'text': 'The Italian Coffee', 'languageCode':...","{'text': 'Cafe', 'languageCode': 'en-US'}"
1,ChIJ8XZ8UABzVo8RtTNBnJ-RwU4,"C. 79ᴮ 534, Itzaes, 97259 Mérida, Yuc., Mexico","{'low': {'latitude': 20.9531456197085, 'longit...","{'text': 'Casa de Pedrito', 'languageCode': 'es'}","{'text': 'Coffee Shop', 'languageCode': 'en-US'}"
2,ChIJ60iXwWlzVo8RQ-Dfk5clkr4,"C. 79E 746A, Nueva Sambulá, 97250 Mérida, Yuc....","{'low': {'latitude': 20.950637719708492, 'long...","{'text': 'Tapiok's', 'languageCode': 'es'}","{'text': 'Ice Cream Shop', 'languageCode': 'en..."
3,ChIJJ91VnOBzVo8Rcw2AXJmECSQ,"97000, SAN SEBASTIAN, 97000 Mérida, Yuc., Mexico","{'low': {'latitude': 20.954222869708495, 'long...","{'text': 'La 81 Pan integral y Café', 'languag...","{'text': 'Coffee Shop', 'languageCode': 'en-US'}"
4,ChIJBRbSQgBzVo8RWXU7n2p37o4,"C. 66 centro, entre 79 La ermita, La Ermita, 9...","{'low': {'latitude': 20.9551800697085, 'longit...","{'text': 'Macchiato Ermita Wine&Coffee', 'lang...","{'text': 'Restaurant', 'languageCode': 'en-US'}"
...,...,...,...,...,...
127,ChIJq3w1S0lxVo8RXeWejN3sd3M,"Calle 18, Av. Rotary Internacional No. 101, It...","{'low': {'latitude': 20.9897345197085, 'longit...","{'text': 'Caffe Latte', 'languageCode': 'en'}","{'text': 'Food', 'languageCode': 'en-US'}"
128,ChIJS3MthclxVo8RiEowTG5a9Xo,"C. 29A 276, Miguel Alemán, 97148 Mérida, Yuc.,...","{'low': {'latitude': 20.9851792197085, 'longit...","{'text': 'Ghost Coffee: Café del Barrio', 'lan...","{'text': 'Coffee Shop', 'languageCode': 'en-US'}"
129,ChIJ7UDD3NxxVo8RZLnVJTluNic,"C. 36 662, Miguel Alemán, 97148 Mérida, Yuc., ...","{'low': {'latitude': 20.990421469708497, 'long...","{'text': 'Komorebi Café', 'languageCode': 'en'}","{'text': 'Restaurant', 'languageCode': 'en-US'}"
130,ChIJKwyCzkpxVo8RSuSbDmGAwG0,"31 #84, Col. México, México, 97125 Mérida, Yuc...","{'low': {'latitude': 20.996231569708495, 'long...","{'text': 'Baguettes Café Circuito', 'languageC...","{'text': 'Restaurant', 'languageCode': 'en-US'}"
